# Dataset & Dataloader
> 데이터세트는 데이터의 집합을 의미하며, 입력값(X)와 결과값(Y)에 대한 정보를 제공하거나 일련의 데이터 묶음을 제공한다.

## 데이터세트 클래스 기본형

- 초기화 메서드( __init__ ): 입력된 데이터의 전처리 과정을 수행

- 호출 메서드( __getitem__ ): 학습을 진행할 때 사용되는 하나의 행을 불러오는 과정

- 길이 반환 메서드( __len__ ): 학습에 사용된 전체 데이터세트의 개수를 반환한다.

In [1]:
class Dataset:
  def __init__(self, data, *arg, **kwargs):
    self.data = data
  
  def __getitem__(self, index):
    return tuple(data[index] for data in data.tensor)
  
  def __len__(self):
    return self.data[0].size(0)

## DataLoader
> Dataset에 저장된 데이터를 어떠한 방식으로 불러와 활용할지 정의한다.

### 제공
- 배치 크기(batch_size): 학습에 사용되는 데이터의 개수가 많이 한 번의 엑포에서 모든 데이터를 메모리에 올릴 수 없을 때 데이터를 나누는 역할을 수행한다.
- 데이터 순서 변경(shuffle): 모델이 데이터 간의 관계가 아닌, 데이터의 순서로 학습되는 것을 방지하고자 수행
- 데이터 로드 프로세스 수(num_workers): 데이터를 불러올 때 사용할 프로세스의 개수를 의미한다.

## 다중 선형 회귀

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

train_x = torch.FloatTensor([
  [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
  [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

train_dataset = TensorDataset(train_x, train_y)
train_dataLoader = DataLoader(train_dataset, batch_size = 2, shuffle= True, drop_last= True)
# drop_last = True : 배치 크기에 맞지 않는 배치를 제거한다. 예를 들어, 데이터세트의 크기가 5일 때, 배치사이즈가 2라면 마지막 배치의 크기는 1이 된다. 1인 마지막 배치를 학습에 포함시키지 않는다.

# 모델, 오차함수, 최적화 선언
model = nn.Linear(2, 2, bias = True)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# 데이터로더 적용
for epoch in range(20000):
  cost = 0.0
  
  for batch in train_dataLoader:
    x, y = batch
    output = model(x)
    
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  
  cost = cost / len(train_dataLoader)
  
  if(epoch + 1) % 1000 == 0:
    print(f'Epoch {epoch + 1:4d} | Model {list(model.parameters())} | Cost: {cost:.3f}')


Epoch 1000 | Model [Parameter containing:
tensor([[ 1.0914, -0.3564],
        [ 0.5474,  0.6114]], requires_grad=True), Parameter containing:
tensor([0.2514, 0.1837], requires_grad=True)] | Cost: 0.082
Epoch 2000 | Model [Parameter containing:
tensor([[ 1.2589, -0.4431],
        [ 0.6909,  0.5371]], requires_grad=True), Parameter containing:
tensor([-0.0028, -0.0340], requires_grad=True)] | Cost: 0.021
Epoch 3000 | Model [Parameter containing:
tensor([[ 1.3444, -0.4872],
        [ 0.7640,  0.4994]], requires_grad=True), Parameter containing:
tensor([-0.1324, -0.1449], requires_grad=True)] | Cost: 0.006
Epoch 4000 | Model [Parameter containing:
tensor([[ 1.3879, -0.5097],
        [ 0.8012,  0.4801]], requires_grad=True), Parameter containing:
tensor([-0.1984, -0.2014], requires_grad=True)] | Cost: 0.001
Epoch 5000 | Model [Parameter containing:
tensor([[ 1.4100, -0.5212],
        [ 0.8202,  0.4703]], requires_grad=True), Parameter containing:
tensor([-0.2320, -0.2302], requires_grad=Tru

# 모델 / 데이터세트 분리

## 모듈 클래스
> 초기화 메서드와 순방향 메서드를 재정의하여 활용한다. 초기화 메서드는 신경망에 사용될 계층을 초기화하고, 순방향 메서드에서는 모델이 어떤 구조를 갖게 될지를 정의한다.

In [3]:
# 모듈 클래스 기본형
class Model(nn.Module):
  def __init__(self):
    super().__init__() # 모듈 클래스의 속성을 초기화
    self.conv1 = nn.Conv2d(1, 20, 5)
    self.conv2 = nn.Conv2d(20, 20, 5)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    return x

In [4]:
# 비선형 회귀

import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x = df.iloc[:, 0].values
    self.y = df.iloc[:, 1].values
    self.length = len(df)
    
  def __getitem__(self, index):
    x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y
  
  def __len__(self):
    return self.length
    
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(2, 1)
    
  def forward(self, x):
    x = self.layer(x)
    return x
    
train_dataset = CustomDataset('../dataset/non_linear.csv')
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle=True, drop_last=True)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
  cost = 0.0
  
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if(epoch+1) % 1000 == 0:
    print(f'epoch: {epoch + 1:4d}, model : {list(model.parameters())}, cost: {cost:.3f}')

    

epoch: 1000, model : [Parameter containing:
tensor([[ 3.1107, -1.7002]], device='mps:0', requires_grad=True), Parameter containing:
tensor([-0.1488], device='mps:0', requires_grad=True)], cost: 0.222
epoch: 2000, model : [Parameter containing:
tensor([[ 3.1097, -1.7028]], device='mps:0', requires_grad=True), Parameter containing:
tensor([-0.0942], device='mps:0', requires_grad=True)], cost: 0.199
epoch: 3000, model : [Parameter containing:
tensor([[ 3.1089, -1.7026]], device='mps:0', requires_grad=True), Parameter containing:
tensor([-0.0442], device='mps:0', requires_grad=True)], cost: 0.196
epoch: 4000, model : [Parameter containing:
tensor([[ 3.1081, -1.7027]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.0017], device='mps:0', requires_grad=True)], cost: 0.171
epoch: 5000, model : [Parameter containing:
tensor([[ 3.1077, -1.7028]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.0435], device='mps:0', requires_grad=True)], cost: 0.168
ep

### 모델 평가

In [6]:
from torch import no_grad
with no_grad():
  model.eval()
  inputs = torch.FloatTensor(
    [
      [1 ** 2, 1],
      [5 ** 2, 5],
      [11 * 2, 11]
    ]
  ).to(device)
  output = model(inputs)
  print(output)

tensor([[ 1.6067],
        [69.3025],
        [49.7713]], device='mps:0')


### 모델 저장

In [9]:
torch.save(
  model,
  "./models/model.pt"
)
torch.save(
  model.state_dict(),
  "./models/model_state_dice.pt"
)

### 데이터세트 분리
> 머신러닝에서 사용되는 전체 데이터세트는 두 가지 쪼는 세 가지로 나눌 수 있다. 전체 데이터 세트는 훈련용 데이터, 테스트 데이터로 분류되고 세분화 하면 검즘용 데이터 까지 분리해 활용한다.

In [10]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

dataset = CustomDataset('../dataset/non_linear.csv')
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Test Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, drop_last=True)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
  cost = 0.0
  
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if(epoch+1) % 1000 == 0:
    print(f'epoch: {epoch + 1:4d}, model : {list(model.parameters())}, cost: {cost:.3f}')
    
with no_grad():
  model.eval()
  for x, y in validation_dataset:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)    
    print(f"X : {x}, Y : {y}, Predict : {output}")

Training Data Size : 160
Validation Data Size : 20
Test Data Size : 20
epoch: 1000, model : [Parameter containing:
tensor([[ 3.1016, -1.7038]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.4096], device='mps:0', requires_grad=True)], cost: 0.080
epoch: 2000, model : [Parameter containing:
tensor([[ 3.1022, -1.7039]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.4672], device='mps:0', requires_grad=True)], cost: 0.076
epoch: 3000, model : [Parameter containing:
tensor([[ 3.0997, -1.7040]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.4898], device='mps:0', requires_grad=True)], cost: 0.075
epoch: 4000, model : [Parameter containing:
tensor([[ 3.0993, -1.7039]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.4987], device='mps:0', requires_grad=True)], cost: 0.077
epoch: 5000, model : [Parameter containing:
tensor([[ 3.0991, -1.7039]], device='mps:0', requires_grad=True), Parameter containing:
t

### 모델 저장 및 불러오기
> 파이토치 모델은 직렬화(Serialize)와 역직렬화(Deserialize)를 통해 객체를 저장하고 불러올 수 있다.

- 모델을 저장하려면 파이썬의 'Pickle'을 활용해 파이썬 객체 구조를 바이너리 프로토콜(Binary Protocols)로 직렬화 한다. 
- 모델을 불러오려면 저장된 객체 파일을 역직렬화해 현재 프로세스의 메모리에 업로드한다.

#### 모델 저장
```python
torch.save(
  model,
  path
)
```

#### 모델 불러오기
```python
torch.load(
  path,
  map_location
)
```
- `map_location` : 모델을 불러올 때 적용하려는 장치 상태를 의미

In [12]:
model = torch.load("./models/model.pt", map_location=device)
print(model)

with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor(
    [
      [1 ** 2, 1],
      [5 ** 2, 5],
      [11 * 2, 11]
    ]
  ).to(device)
  output = model(inputs)
  print(output)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)
tensor([[ 1.6067],
        [69.3025],
        [49.7713]], device='mps:0')


In [13]:
# 모델 구조 확인하기
import torch
from torch import nn

class CustomModel(nn.Module):
  pass

model = torch.load("./models/model.pt", map_location=device)
print(model)


CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


### 모델 상태 저장 및 불러오기
> 모델 전체를 저장하면 모델의 모든 모든 정보를 저장하므로 모델 상태만 저장하는 것보다 더 많은 저장공간이 필요하다. 그러므로 모델의 매개변수만 저장하여 활용하는 방법을 사용할 수 있다.

```python
torch.save(
  model.state_dict(),
  path
)
```
> 이러한 방식으로 저장된 모델에는 학습된 CustomModel 객체의 가중치와 편향이 저장돼 있다. 즉, 추론에 필요한 데이터만 가져와 저장하는 방식으로 이해할 수 있다.

### 체크포인트
> 체크포인트는 학습 과정의 특정 지점마다 저장하는 것을 의미한다. 학습 시 예기치 못하게 오류가 발생하거나 시스템 리소스 과부하 등으로 학습니 정상적으로 마무리 되지 않을경우 활용할 수 있다.
